# 기업 시장/재무 데이터 분석 

## 1) 기업 종목 코드 크롤링 

### 1-1) 한국거래소 데이터를 바로 크롤링

밑의 코드를 사용하면 현재 한국거래소에 상장되어있는 기업들의 기업명, 종목코드를 간편하게 수집 가능.  
하지만 이 방법으로는 업종코드를 따로 붙여야함.  

[ 참고: http://excelsior-cjh.tistory.com/109 ] 

In [13]:
# 필요한 모듈 임포트 

import re
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [14]:
# 한국거래소에서 상장기업정보 받기 

import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명','종목코드']]
code_df = code_df.rename(columns={'회사명':'name','종목코드':'code'})

code_df.head()

,name,code
0,CJ,001040
1,HDC,012630
2,HSD엔진,082740
3,KG케미칼,001390
4,LG이노텍,011070


### 1-2) 한국거래소에서 csv 파일 다운로드 

한국거래소에서 다운받은 상장기업정보 csv파일을 dataframe으로 읽어옴.  
업종코드가 붙어있어서 편하지만, 직접 csv파일을 다운받아야하는 번거로움.  

In [15]:
# 한국거래소에서 다운받은 csv파일 데이터프레임으로 저장하기 
#-*- coding: utf-8 -*-

corp_df = pd.read_csv('corp_code2.csv')

# 종목코드 6자리 문자열로 맞추기 

adj_code = []

for i in corp_df.code:
     adj_code.append(str(i).zfill(6))

corp_df.code = adj_code

# csv 다운로드 

corp_df.to_csv('corp_df.csv')

In [16]:
corp_df.head()

,code,name,cate_code,cate_name
0,060310,3S,32902,특수 목적용 기계 제조업
1,095570,AJ네트웍스,147603,산업용 기계 및 장비 임대업
2,068400,AJ렌터카,147601,운송장비 임대업
3,006840,AK홀딩스,116409,기타 금융업
4,054620,APS홀딩스,116409,기타 금융업


## 2) 기업 재무 데이터 크롤링 

네이버 금융 페이지 웹크롤링 

[ 참고: https://financedata.github.io/posts/naver-finance-finstate-crawling.html ]



In [17]:
# 컬럼명 정리 함수 생성 

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()
        date_str = date_str.replace('/', '-')
    return date_str

In [18]:
def get_finstate_naver(code, fin_type='0', freq_type='Q'):

    # 종목코드에 따른 url 생성 
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                   'cmp_cd=%s&fin_typ=%s&freq_typ=%s'
    url = url_tmpl % (code, fin_type, freq_type)

    # 재무정보 크롤링 
    dfs = pd.read_html(url, encoding="utf-8")
    df = dfs[0]
    df = df.set_index('주요재무정보')
    
    # 컬럼명 정리 
    cols = list(df.columns)
    cols = [get_date_str(cols[i][1]) for i in range(len(cols))]
    df.columns = cols
    df = df.ix[:, :-1]

    # 전치행렬
    dft = df.T
    
    # 컬럼명 정리 2    
    col = list(dft.columns)
    adj_col = [dft.columns[i][0] for i in range(len(col))]
    dft.columns = adj_col

    # date 데이터타입 변경 
    dft.index = pd.to_datetime(dft.index)

    # remove if index is NaT
    dft = dft[pd.notnull(dft.index)]
#    dft = dft[pd.notnull(dft.매출액)]
    
    # corp_df에서 코드명 컬럼 가져오기 
    dft['code'] = code
    
    # 컬럼명 영어로 변경 
    dft = dft.rename(columns={'매출액': 'sales', 
                              '영업이익':'operating revenue', 
                              '영업이익(발표기준)':'operating revenue(announced)', 
                              '세전계속사업이익':'earnings before tax', 
                              '당기순이익':'net income', 
                              '당기순이익(지배)':'net income attribuable to owners' ,     
                              '당기순이익(비지배)':'net income attributable to non-controlling interests', 
                              '자산총계':'assets',
                              '부채총계':'liabilities', 
                              '자본총계':'equity', 
                              '자본총계(지배)':'owners equity', 
                              '자본총계(비지배)':'non-controlling equity', 
                              '자본금':'capital stock',
                              '영업활동현금흐름':'cash flows from operating activities',
                              '투자활동현금흐름':'cash flows from investing activities', 
                              '재무활동현금흐름':'cash flows from financing activities', 
                              #'CAPEX', 
                              #'FCF', 
                              '이자발생부채':'interests-bearing liabilities', 
                              '영업이익률':'ratio of operating profit to revenue',
                              '순이익률':'ROS', 
                              'ROE(%)':'ROE', 
                              'ROA(%)':'ROA', 
                              '부채비율':'debt ratio', 
                              '자본유보율':'capital-holding ratio', 
                              'EPS(원)':'EPS', 
                              'PER(배)':'PER',
                              'BPS(원)':'BPS', 
                              'PBR(배)':'PBR', 
                              '현금DPS(원)':'cash DPS', 
                              '현금배당수익률':'cash dividend yield ratio', 
                              '현금배당성향(%)':'cash dividend payout ratio', 
                              '발행주식수(보통주)':'total issued stock',
                              #'code'
                             })
    
    
    return dft


get_finstate_naver('000120')

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


,sales,operating revenue,operating revenue(announced),earnings before tax,net income,net income attribuable to owners,net income attributable to non-controlling interests,assets,liabilities,equity,...,capital-holding ratio,EPS,PER,BPS,PBR,cash DPS,cash dividend yield ratio,cash dividend payout ratio,total issued stock,code
2017-06-01,15949.0,511.0,511.0,129.0,44.0,38.0,7.0,54580.0,27468.0,27112.0,...,2449.30,165.0,105.88,133766.0,1.24,NaN,NaN,NaN,22812344.0,000120
2017-09-01,17078.0,619.0,619.0,379.0,245.0,223.0,22.0,56884.0,29305.0,27579.0,...,2468.88,978.0,125.05,135472.0,1.33,NaN,NaN,NaN,22812344.0,000120
2017-12-01,18732.0,626.0,626.0,204.0,108.0,93.0,15.0,63145.0,34873.0,28273.0,...,2476.98,406.0,83.13,136148.0,1.20,NaN,NaN,NaN,22812344.0,000120
2018-03-01,19344.0,600.0,600.0,19.0,-9.0,-38.0,30.0,63089.0,35262.0,27827.0,...,2474.00,-169.0,101.44,134556.0,1.04,NaN,NaN,0.0,22812344.0,000120
2018-06-01,20015.0,453.0,453.0,434.0,348.0,329.0,19.0,72386.0,42464.0,29922.0,...,2514.50,1443.0,51.73,138708.0,0.99,NaN,NaN,0.0,22812344.0,000120
2018-09-01,21351.0,551.0,NaN,226.0,132.0,209.0,NaN,NaN,NaN,NaN,...,NaN,916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000120
2018-12-01,22414.0,644.0,NaN,294.0,182.0,199.0,NaN,NaN,NaN,NaN,...,NaN,874.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000120


In [19]:
# 2208개의 상장기업 재무정보 데이터프레임 union join

fin_df = pd.DataFrame()

for i in list(corp_df.code):
    fin_df = pd.concat([fin_df,get_finstate_naver(i)])
    fin_df = fin_df[pd.notnull(fin_df.sales)]

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [20]:
len(fin_df)

10767

In [21]:
# csv로 저장 

fin_df.to_csv('fin_df.csv', encoding='utf-8') # 다운은 받아지나 csv 열었을 때 한글컬럼명 깨지는 문제  

## 3) 기업 시장 데이터 크롤링 

### 3-1) 야후 모듈 사용하여 일별 시세 크롤링 

하지만 누락되는 데이터가 보여서 코드만 첨부

[ 참고: https://gomjellie.github.io/%ED%8C%8C%EC%9D%B4%EC%8D%AC/pandas/%EC%A3%BC%EC%8B%9D/2017/06/09/pandas-datareader-stock.html ] 



In [ ]:
from pandas_datareader import data
import fix_yahoo_finance as yf
yf.pdr_override()
start_date = '2018-07-01'
tickers = ['067160.KQ', '035420.KS','000660.KS']
afreeca = data.get_data_yahoo(tickers[0], start_date)
naver = data.get_data_yahoo(tickers[1], start_date)
hynix = data.get_data_yahoo(tickers[2], start_date)

naver

### 3-2) 네이버 금융 사이트에서 일별 시세 크롤링 



In [ ]:
# 일별시세 크롤링 함수 생성

def get_market_price(code) :
    price_df = pd.DataFrame()
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=' + str(code)
    for page in range(1,3) : 
        url = url_tmpl + '&page=' + str(page)
        df = pd.read_html(url, header=0)[0]
        price_df = pd.concat([price_df,df])
        price_df = price_df.dropna()
    price_df['code'] = code
    return price_df


get_market_price('054620')
    

In [ ]:
# 2208개 상장기업 일별시세 합치기 

price_df = pd.DataFrame()

for i in list(corp_df.code) :
    price_df = pd.concat([price_df,get_market_price(i)])

In [ ]:
len(price_df)

In [ ]:
price_df.to_csv('price_df.csv')

## 2-1) 재무데이터 변수 가공 

- 크롤링한 재무정보를 활용하여 수익성, 안정성, 성장성, 활동성을 의미하는 재무지표를 생성 
    - 재무지표는 네이버 금융 참고 
    - *현재는 네이버 금융 기업현황에 있는 기본적인 재무지표만 활용하여 변수를 가공하지만, 추후에는 재무상태표, 포괄손익계산서, 현금흐름표 모두 크롤링해서 좀 더 다양한 변수 활용 가능*



![image](https://user-images.githubusercontent.com/28600272/43033950-4221ba8e-8d0e-11e8-9dc3-3a57d177e8b4.png)

In [49]:
#### 성장률 변수를 위해 전기, 당기 재무정보만 사용 

before = '2017-12-01'
after = '2018-03-01'

### 새로 가공한 변수를 담기위한 데이터프레임 생성 

#new_df = pd.DataFrame()
new_df['code']=corp_df.code


,code,sales_qoq,operating_revenue_qoq,net_income_qoq,assets_qoq,equity_qoq
0,060310,0.277778,-3.400000,-1.500000,0.042000,0.010753
1,095570,-0.002962,-0.298077,-0.951807,0.043251,0.002691
2,068400,-0.042771,-0.406250,-0.843137,-0.005071,0.002442
3,006840,0.052103,-0.370495,-0.661039,0.014783,0.014977
4,054620,-1.625418,-4.111111,-25.571429,-0.109147,-0.036278
5,265520,-0.158924,-9.714286,-24.000000,0.015625,-0.127680
6,211270,0.797468,3.875000,0.000000,0.077963,0.018610
7,027410,0.518116,0.172414,34.373932,-0.613207,-1.280260
8,282330,0.000000,0.000000,0.000000,0.000000,0.000000
9,138930,0.777833,-1.348996,-1.468401,-0.018150,-0.016962


In [89]:
test = new_df

In [97]:
#### 1) 성장성, 안정성, 가치지표를 위한 리스트 생성 


temp_list = list()

temp_columns = ['ratio of operating profit to revenue','ROS','ROE','ROA', # 수익성
                'debt ratio', # 안정성
                'EPS', 'BPS', 'PER', 'EPS', 'PBR' ]#가치지표
            

for c in temp_columns:
    temp_list = list()
    for i in new_df.code: 
        try : 
            temp_list.append((fin_df[fin_df['code']==i][c][after]-fin_df[fin_df['code']==i][c][before])/fin_df[fin_df['code']==i][c][before])
        except:
            temp_list.append(0)
    test[c]= temp_list

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [47]:
### 2) 성장성 ################지표를 위한 리스트 생성 

sales_qoq_list = list()
operating_revenue_qoq_list = list()
net_income_qoq_list = list()
assets_qoq_list = list()
equity_qoq_list = list()


for i in new_df.code:
    try:
        sales_qoq_list.append((fin_df[fin_df['code']==i]['sales'][after]-fin_df[fin_df['code']==i]['sales'][before])/fin_df[fin_df['code']==i]['sales'][before])
    except:
        sales_qoq_list.append(0)

for i in new_df.code:
    try:
        operating_revenue_qoq_list.append((fin_df[fin_df['code']==i]['operating revenue'][after]-fin_df[fin_df['code']==i]['operating revenue'][before])/fin_df[fin_df['code']==i]['operating revenue'][before])
    except:
        operating_revenue_qoq_list.append(0)

for i in new_df.code:
    try:
        net_income_qoq_list.append((fin_df[fin_df['code']==i]['net income'][after]-fin_df[fin_df['code']==i]['net income'][before])/fin_df[fin_df['code']==i]['net income'][before])
    except:
        net_income_qoq_list.append(0)
        
for i in new_df.code:
    try:
        assets_qoq_list.append((fin_df[fin_df['code']==i]['assets'][after]-fin_df[fin_df['code']==i]['assets'][before])/fin_df[fin_df['code']==i]['assets'][before])
    except:
        assets_qoq_list.append(0)
        
for i in new_df.code:
    try:
        equity_qoq_list.append((fin_df[fin_df['code']==i]['equity'][after]-fin_df[fin_df['code']==i]['equity'][before])/fin_df[fin_df['code']==i]['equity'][before])
    except:
        equity_qoq_list.append(0)
        
        
        
         
new_df['sales_qoq']=sales_qoq_list        
new_df['operating_revenue_qoq']=operating_revenue_qoq_list
new_df['net_income_qoq']=net_income_qoq_list
new_df['assets_qoq']=assets_qoq_list
new_df['equity_qoq']=equity_qoq_list


new_df  

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


,code,sales_qoq,operating_revenue_qoq,net_income_qoq,assets_qoq,equity_qoq
0,060310,0.277778,-3.400000,-1.500000,0.042000,0.010753
1,095570,-0.002962,-0.298077,-0.951807,0.043251,0.002691
2,068400,-0.042771,-0.406250,-0.843137,-0.005071,0.002442
3,006840,0.052103,-0.370495,-0.661039,0.014783,0.014977
4,054620,-1.625418,-4.111111,-25.571429,-0.109147,-0.036278
5,265520,-0.158924,-9.714286,-24.000000,0.015625,-0.127680
6,211270,0.797468,3.875000,0.000000,0.077963,0.018610
7,027410,0.518116,0.172414,34.373932,-0.613207,-1.280260
8,282330,0.000000,0.000000,0.000000,0.000000,0.000000
9,138930,0.777833,-1.348996,-1.468401,-0.018150,-0.016962
